In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sbs

In [30]:
url = "http://slnxdesa01:4998/Otrs/ObtenerListaTicket"

response = requests.get(url)

#Verificar el codifo del estado de la respuesta
if response.status_code == 200:
    data = response.json()
    
    
    reporte_otrs = pd.DataFrame(data)
    
else:
    print("Error al crear dataframe de ")
    

del response, data,url

In [31]:
#renombramos las columnas
reporte_otrs = reporte_otrs.rename(
    columns = {
        "ticket": "Ticket#",
        "titulo": "Titulo",
        "cola": "Cola",
        "estado": "Titulo",
        "cliente": "Cliente",
        "fecha": "Fecha",
        "creadoPor": "Creado Por",
        "tiempo_Trabajado": "Tiempo Trabajado",
        "downtime": "Downtime",
        "alcance": "Alcance",
        "cola_Principal": "Cola Principal",
        "cola_Secundaria": "Titulo",
        "tiempo_Trabajado_Horas": "Tiempo Trabajado (Horas)",
        "downtime_Horas": "Downtime (Horas)",
    }
)

In [32]:
#separo la columna para solamente tener la fecha sin la hora ni la region
#se dividen las columnas y obtengo la posicion 0
reporte_otrs["Fecha"] = reporte_otrs["Fecha"].str.split(' ').str[0]
reporte_otrs["Fecha"] = pd.to_datetime(reporte_otrs["Fecha"])

In [33]:
#creando downtime de cada cola principal desglosada

# Track3
track_downtime = reporte_otrs[reporte_otrs["Cola Principal"]=="Track3"].groupby("Cola",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending = False).loc[lambda x : x["Downtime"] > 0]

# BackOffice
backoffice_downtime = reporte_otrs[reporte_otrs["Cola Principal"] == "BackOffice"].groupby("Cola",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False).loc[lambda x: x["Downtime"] > 0]

# Operaciones Coto Digital
ncd_downtime = reporte_otrs[reporte_otrs["Cola Principal"] == "Operaciones Coto Digital"].groupby("Cola",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False).loc[lambda x:x["Downtime"]>0]

In [34]:
#downtime por Cola Principal
downtime_por_cola_principal = reporte_otrs.groupby("Cola Principal",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False)

#downtime por Cola
downtime_por_cola = reporte_otrs.groupby("Cola",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False)

In [35]:
#cantidad de tickets por Cola Principal
conteo_tickets_por_cola_principal = reporte_otrs.groupby("Cola Principal", as_index=False)["Ticket#"].count().sort_values(by="Ticket#",ascending=False)

#cantidad de tickets por Cola
conteo_tickets_por_cola = reporte_otrs.groupby("Cola",as_index=False)["Ticket#"].count()


In [36]:
#tiempo trabajado por cola principal
tiempo_trabajado_por_cola_principal = reporte_otrs.groupby("Cola Principal",as_index=False)["Tiempo Trabajado"].sum().sort_values(by="Tiempo Trabajado",ascending=False).loc[lambda x : x["Tiempo Trabajado"]> 0]

#tiempo trabajado por cola
tiempo_trabajado_por_cola = reporte_otrs.groupby("Cola",as_index=False)["Tiempo Trabajado"].sum().sort_values(by="Tiempo Trabajado",ascending=False).loc[lambda x : x["Tiempo Trabajado"]> 0]

In [45]:
#creamos todas las agrupaciones de "Cola Principal" necesarias en una tabla
dynamics_cola_principal = reporte_otrs.groupby('Cola Principal').agg(
    Downtime_Minutos=('Downtime', 'sum'),# Suma de la columna 'Valor' como 'downtime'
    Downtime_Horas=('Downtime (Horas)','sum'),
    Tiempo_Trabajado_Minutos =  ('Tiempo Trabajado', 'sum'),
    Tiempo_Trabajado_Horas = ('Tiempo Trabajado (Horas)','sum'),
    Cantidad_Tickets=('Ticket#', 'count')   # Conteo de la columna 'Valor' como 'ticket'
).reset_index()

dynamics_cola_principal

,Cola Principal,Downtime_Minutos,Downtime_Horas,Tiempo_Trabajado_Minutos,Tiempo_Trabajado_Horas,Cantidad_Tickets
0,AIX,216,3.600,23,0.384,6
1,Administracion NT,76,1.266,54,0.900,4
2,Administracion de la operacion Coto Digital,115,1.917,1519,25.311,296
3,Administracion de la operación Coto Digital,0,0.000,60,1.000,1
4,BackOffice,198,3.301,5953,99.200,769
5,Base de Datos,278,4.634,185,3.083,5
6,Comerciales,0,0.000,626,10.431,39
7,Comunicaciones,805,13.417,93,1.550,4
8,Multistore,459,7.650,1796,29.926,89
9,Operaciones Coto Digital,588,9.800,245,4.083,26


In [43]:
#creamos todas las agrupaciones de "Cola"
dynamics_cola = reporte_otrs.groupby('Cola').agg(
    Downtime_Minutos=('Downtime', 'sum'),# Suma de la columna 'Valor' como 'downtime'
    Downtime_Horas=('Downtime (Horas)','sum'),
    Tiempo_Trabajado_Minutos =  ('Tiempo Trabajado', 'sum'),
    Tiempo_Trabajado_Horas = ('Tiempo Trabajado (Horas)','sum'),
    Cantidad_Tickets=('Ticket#', 'count')   # Conteo de la columna 'Valor' como 'ticket'
).reset_index()

dynamics_cola

,Cola,Downtime_Minutos,Downtime_Horas,Tiempo_Trabajado_Minutos,Tiempo_Trabajado_Horas,Cantidad_Tickets
0,AIX,215,3.583,21,0.351,5
1,AIX::Causa: Operador de Computos no visualiza ...,1,0.017,2,0.033,1
2,Administracion NT,76,1.266,54,0.900,4
3,Administracion de la operacion Coto Digital::C...,0,0.000,82,1.366,3
4,Administracion de la operacion Coto Digital::C...,0,0.000,30,0.500,2
...,...,...,...,...,...,...
111,Track3,81,1.351,534,8.899,14
112,Track3::APk Carga,10,0.167,3,0.050,1
113,Track3::APk Cierre,0,0.000,15,0.250,1
114,Track3::APk PreCarga,28,0.467,22,0.367,3
